# Setup project

In [2]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
# Pkg.add("DataLoaders")
# Pkg.add("Plots")
# Pkg.add("CUDA")
# Pkg.add("Distributions")

  Activating project at `g:\桌面\2022 Fall\cs268\final_proj\Project-Optimizer`


In [3]:
using CSV
using DataLoaders
using Plots
using CUDA
using Distributions 
using LinearAlgebra
using Statistics
using Printf

# Create a custom ML structure to apply custom optimizers.

## Prepare data

The Iris dataset is from UC Irvine Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/iris

### Create a one-hot encoder

In [4]:
function one_hot(y)
    rslt = zeros(3)
    rslt[trunc(Int, y) + 1] = 1.0
    return rslt
end

one_hot (generic function with 1 method)

### Read all lines of data in train and test

Iris dataset

In [5]:
# train and test
train_x_data = []
train_y_data = []
# 8:2
# test_x_data = []
# test_y_data = []
for line in eachline("iris.txt")
    splitted = collect(split(line, " "))
    push!(train_x_data, [parse(Float64, splitted[4]), parse(Float64, splitted[7]), parse(Float64, splitted[10]), parse(Float64, splitted[13])])
    push!(train_y_data, one_hot(parse(Float64, splitted[16])))
end

Inspect shape of train and test

In [6]:
print(size(train_x_data))
print(size(train_y_data))

(148,)(148,)

### Create dataloader

In [7]:
batch_size = 33
dl_train = DataLoader((train_x_data, train_y_data), batch_size)
# dl_test = DataLoader((test_x, test_y), batch_size)

DataLoaders.GetObsParallel{DataLoaders.BatchViewCollated{Tuple{Vector{Any}, Vector{Any}}}}(batchviewcollated() with 5 batches of size 33, false)

## Create whole machine learning pipeline

### Create an activation function: ReLU 

In [8]:
function Relu!(Xs)
    for i = 1 : length(Xs)
        Xs[i] = max(0.0,Xs[i])
    end
end

Relu! (generic function with 1 method)

### Create fully connected layers

In [9]:
# Xs in only 1 set of input in a batch.
function fc(Xs, num_in_channel, num_out_channel, W)
    rslt = zeros(num_out_channel)
    Xs = reshape(Xs, 1, num_in_channel)
    b = rand()
    for i = 1:num_out_channel
        temp = reshape(W[:, i], num_in_channel, 1)
        rslt[i] = (Xs * temp)[1]+b
    end
    return rslt
end

fc (generic function with 1 method)

### Create a simple MLP model

In [10]:
# Xs in only 1 set of input in a batch.
function MLP(Xs, out_channel; num_hidden_layers = 0, hidden_layer_size = 64, Ws = nothing)
    Ws_idx = 1
    hidden_outs = []
    # layer in
    if Ws == nothing
        Wss = []
        W = rand(Uniform(-0.1, 0.1), size(Xs)[1], hidden_layer_size)
        push!(Wss, W)
    else
        W = Ws[Ws_idx]
        Ws_idx += 1
    end
    out = fc(Xs, size(Xs)[1], hidden_layer_size, W)
    Relu!(out)
    push!(hidden_outs, deepcopy(out))
    # layer hidden
    for i = 1 : num_hidden_layers
        if Ws == nothing
            W = rand(Uniform(-0.1, 0.1), hidden_layer_size, hidden_layer_size)
            push!(Wss, W)
        else
            W = Ws[Ws_idx]
            Ws_idx += 1
        end
        out = fc(out, hidden_layer_size, hidden_layer_size, W)
        Relu!(out)
        push!(hidden_outs, deepcopy(out))
    end
    # layer out
    if Ws == nothing
        W = rand(Uniform(-0.1, 0.1), hidden_layer_size, out_channel)
        push!(Wss, W)
    else
        W = Ws[Ws_idx]
        Ws_idx += 1
    end
    out = fc(out, hidden_layer_size, out_channel, W)
    if Ws == nothing
        return out, hidden_outs, Wss
    end
    return out, hidden_outs
end

MLP (generic function with 1 method)

### Create a loss function: soft-max loss
Soft-max loss is a combination of a soft-max activation layer and cross entropy loss

In [11]:
# Y_pred in only 1 set of prediction in a batch. Ys in only 1 set of truth label in a batch.
function softmax_loss(Y_pred, Ys)
    # softmax
    temp_sum = 0.0
    for x in Y_pred
        temp_sum += exp(x)
    end
    l = length(Y_pred)
    scores = zeros(l)
    for i = 1:l
        scores[i] = exp(Y_pred[i])/temp_sum
    end
    # loss
    losses = zeros(l)
    for i = 1:l
        losses[i] = -Ys[i] * log(scores[i])
    end
    total_loss = sum(losses)
    
    return scores, losses, total_loss
end

softmax_loss (generic function with 1 method)

How many threads we have for cpu?

In [12]:
Threads.nthreads()

32

### Create a back-propagation procedure

In [156]:
#Ws[idx of layer][idx of node, idx of weights]
function ∇(Xs, Ys, Ws, scores, outs, hidden_outs)
    num_in_channel = size(Xs)[1]
    num_out_channel = size(Ys)[1]
    hidden_layer_size = size(hidden_outs[1])[1]
    num_hidden_layers = size(hidden_outs)[1]
    # step -2
    ∂loss_∂netout = zeros(num_out_channel)
    for i = 1:num_out_channel
        ∂loss_∂netout[i] = -Ys[i]/scores[i]
    end
    # println("step -2: $∂loss_∂netout")
    # step -1
    outs_exp_sum = 0.0
    for out in outs
        outs_exp_sum += exp(out)
    end
    # println("step -1: $outs_exp_sum")
    for i = 1:num_out_channel
        ∂loss_∂netout[i] *= (exp(outs[i])*(outs_exp_sum - exp(outs[i])))/(outs_exp_sum^2)
    end
    # println("step -1: $∂loss_∂netout")
    # Assume we use same hidden_size of all hidden layers for the sake for brevity
    ∇s = []
    # first layer
    push!(∇s, zeros(num_in_channel, hidden_layer_size))
    # hidden layer
    for i = 1 : num_hidden_layers-1
        push!(∇s, zeros(hidden_layer_size, hidden_layer_size))
    end
    # last layer
    push!(∇s, zeros(hidden_layer_size, num_out_channel))

    # Start back_propagations
    ∂loss_∂prev_nodes = zeros(hidden_layer_size)
    # layer weights from outs to last of hidden layer
    for i = 1 : hidden_layer_size
        ∂loss_∂prev_nodes[i] = back_propagation(hidden_outs[num_hidden_layers][i],
        num_hidden_layers+1, i, ∇s,
        Ws[num_hidden_layers+1][i, :], ∂loss_∂netout)
    end
    # layer weights from i-th hidden layer to i-1th hidden layer
    for k = num_hidden_layers : -1 : 2
        ∂loss_∂curr_nodes = deepcopy(∂loss_∂prev_nodes)
        for i = 1 : hidden_layer_size
            ∂loss_∂prev_nodes[i] = back_propagation(hidden_outs[k-1][i],
            k, i, ∇s,
            Ws[k][i, :], ∂loss_∂curr_nodes)
        end
    end
    # layer weights from 1st hidden layer to input layer
    for i = 1 : num_in_channel
        back_propagation(Xs[i],
        1, i, ∇s,
        Ws[1][i, :], ∂loss_∂prev_nodes)
    end
    return ∇s
end

∇ (generic function with 1 method)

In [14]:
function back_propagation(node_out, layer_idx, i, ∇s, weights, ∂loss_∂nodes)
    ∂loss_∂node_i = (node_out==0) ? 0 : dot(weights, ∂loss_∂nodes) # considering d of ReLU
    ∇s[layer_idx][i, :] = ∂loss_∂nodes*node_out
    return ∂loss_∂node_i
end

back_propagation (generic function with 1 method)

### Create a training procedure

In [260]:
function batch_step(optimizer, Xs_batch, Ys_batch, Ws_wrapped; hidden_layer_size = 64)
    b_s = size(Xs_batch)[2]
    t = typeof(optimizer)
    # b_s = 33
    cache_new_Ws_unwrapped = zeros(b_s, num_Ws)
    total_losses = zeros(b_s)
    Ws_unwrapped = _unwrap(Ws_wrapped)
    optimizer.k = optimizer.k + 1
    Threads.@threads for i in 1:b_s-1
        Xs = Xs_batch[:, i]
        Y_truth = one_hot(Ys_batch[i])
        #forward
        Y_pred, hidden_outs = MLP(Xs, 3; num_hidden_layers = 2, Ws=Ws_wrapped, hidden_layer_size = hidden_layer_size)
        scores, losses, total_loss = softmax_loss(Y_pred, Y_truth)
        total_losses[i] = total_loss
        #backward
        if  t <: NesterovMomentum_SGD || t <: Nelder_Mead
            cache_new_Ws_unwrapped[i,:] = step_without_update!(optimizer, Ws_unwrapped, Xs, Y_truth)
        else
            gradients = ∇(Xs, Y_truth, Ws_wrapped, scores, Y_pred, hidden_outs);
            if t <: LBFGS || t <: ConjugateGD
                cache_new_Ws_unwrapped[i,:] = 
                step_without_update!(optimizer, _unwrap(gradients), Ws_unwrapped, Xs, Y_truth)
            else
                cache_new_Ws_unwrapped[i,:] = step_without_update!(optimizer, _unwrap(gradients), Ws_unwrapped)
            end
        end
        # if i==1
        #     println()
        #     println(Y_pred)
        #     println(scores)
        #     println(Y_truth)
        #     println()
        # end
    end
    i = b_s
    Xs = Xs_batch[:, i]
    Y_truth = one_hot(Ys_batch[i])
    #forward
    Y_pred, hidden_outs = MLP(Xs, 3; num_hidden_layers = 2, Ws=Ws_wrapped, hidden_layer_size = hidden_layer_size)
    scores, losses, total_loss = softmax_loss(Y_pred, Y_truth)
    total_losses[i] = total_loss
    #backward
    gradients = ∇(Xs, Y_truth, Ws_wrapped, scores, Y_pred, hidden_outs);
    if  t <: NesterovMomentum_SGD || t <: Nelder_Mead
        cache_new_Ws_unwrapped[i,:] = step!(optimizer, Ws_unwrapped, Xs, Y_truth)
    else
        gradients = ∇(Xs, Y_truth, Ws_wrapped, scores, Y_pred, hidden_outs);
        if t <: LBFGS || t <: ConjugateGD
            cache_new_Ws_unwrapped[i,:] = 
            step!(optimizer, _unwrap(gradients), Ws_unwrapped, Xs, Y_truth)
        else
            cache_new_Ws_unwrapped[i,:] = step!(optimizer, _unwrap(gradients), Ws_unwrapped)
        end
    end

    #update parameters
    # println(cache_new_Ws_unwrapped[:,1])
    new_Ws_unwrapped = mean(cache_new_Ws_unwrapped, dims=1)
    # println(new_Ws_unwrapped[1])
    # println(" loss = $(mean(total_losses))")
    # println(" mean new para = $(mean(new_Ws_unwrapped))")
    return _wrap(new_Ws_unwrapped, params_shape), mean(total_losses)
end

batch_step (generic function with 1 method)

In [16]:
function epoch_step(optimizer, new_Ws_wrapped; hidden_layer_size = 64)
    ct_batch = 1
    curr_loss = 0
    for (xs, ys) in dl_train
        print("batch# $ct_batch --> ")
        ct_batch += 1
        new_Ws_wrapped, curr_loss = batch_step(optimizer, xs, ys, new_Ws_wrapped; hidden_layer_size = hidden_layer_size)
        flush(stdout)
    end
    println("\n\ttrain loss =  $curr_loss")
    return new_Ws_wrapped, curr_loss
end

epoch_step (generic function with 1 method)

# Create optimizers

#### Helper functions to make inputs compatible with optimizers

In [17]:
function _unwrap(wrapped)
    unwrapped = []
    for layer in wrapped
        unwrapped = vcat(unwrapped, reduce(vcat,layer))
    end
    return unwrapped
end
function _wrap(unwrapped, shape)
    wrapped = []
    curr_idx = 1
    for s in shape
        ct = s[1]*s[2]
        push!(wrapped, Float64.(reshape(unwrapped[curr_idx:curr_idx-1+ct], s)))
        curr_idx += ct
    end
    return wrapped
end

_wrap (generic function with 1 method)

#### A simple and fast line search algorithm by David

In [285]:
function line_serach(F, d, x, train_x, train_y; r=0.5,c=1e-4,nmax=10)
    
    # params
    # F: function to be optimized
    # x: variable
    # d: direction
    # r: factor by which to reduce step size at each iteration
    # c: parameter [0,1]
    # nmax: max iteration

    # return
    # α step size
    # fk1: function value at new x
    # gkk: gradient at new x

    #https://en.wikipedia.org/wiki/Backtracking_line_search
    α=1

    fk, hidden_outs, Y_pred, scores = F(train_x, train_y, x)
    # println("(train_x,train_y,x,scores,Y_pred,hidden_outs)=>
    # [$(size(train_x))],[$(size(train_y))],[$(size(x))],[$(size(scores))],[$(size(Y_pred))],[$(size(hidden_outs))]")
    gk = _unwrap(∇(train_x, train_y, _wrap(x, params_shape), scores, Y_pred, hidden_outs))
    # fk,gk=F(x)

    xx=x
    x=x+α*d

    fk1, hidden_outs, Y_pred, scores = F(train_x, train_y, x) 
    gk1 = _unwrap(∇(train_x, train_y, _wrap(x, params_shape), scores, Y_pred, hidden_outs))
    # fk1,gk1=F(x)
    n=1
    
    while fk1>fk+c*α*(gk'*d) && n < nmax
        n=n+1
        α=α*0.5
        x=xx+α*d

        fk1, hidden_outs, Y_pred, scores = F(train_x, train_y, x) 
        gk1 = _unwrap(∇(train_x, train_y, _wrap(x, params_shape), scores, Y_pred, hidden_outs))
        # fk1,gk1=F(x)
    end
    if α == 1
        α = -1
        while fk1>fk+c*α*(gk'*d) && n < nmax
            n=n+1
            α=α*0.5
            x=xx+α*d
    
            fk1, hidden_outs, Y_pred, scores = F(train_x, train_y, x) 
            gk1 = _unwrap(∇(train_x, train_y, _wrap(x, params_shape), scores, Y_pred, hidden_outs))
            # fk1,gk1=F(x)
        end
    end
    return α, fk1, gk1
end

line_serach (generic function with 1 method)

#### Steepest gradient descent
Credit: Wenbo

In [18]:
abstract type DescentMethod end
# Adam from K&W page 81
mutable struct SteepestGD <: DescentMethod
    α # learning rate
    k # dumb variable
end
function init!(M::SteepestGD)
    M.k = 0
end
function step!(M::SteepestGD, ∇f, x)
    return x - M.α*∇f
end
function step_without_update!(M::SteepestGD, ∇f, x)
    return x - M.α*∇f
end

step_without_update! (generic function with 1 method)

#### Adam
Credit: Wenbo

In [19]:
abstract type DescentMethod end
# Adam from K&W page 81
mutable struct Adam <: DescentMethod
    α # learning rate
    γv # decay
    γs # decay
    ϵ # small value
    k # step counter
    v # 1st moment estimate
    s # 2nd moment estimate
end
function init!(M::Adam, x_length)
    M.k = 0
    M.v = zeros(x_length)
    M.s = zeros(x_length)
    return M
end
function step!(M::Adam, ∇f, x)
    α, γv, γs, ϵ, k = M.α, M.γv, M.γs, M.ϵ, M.k
    s, v, g = M.s, M.v, ∇f
    M.v = γv*v + (1-γv)*g
    M.s = γs*s + (1-γs)*g.*g
    # M.k = k += 1
    v_hat = M.v ./ (1 - γv^k)
    s_hat = M.s ./ (1 - γs^k)
    return x - α*v_hat ./ (sqrt.(s_hat) .+ ϵ)
end
function step_without_update!(M::Adam, ∇f, x)
    α, γv, γs, ϵ, k = M.α, M.γv, M.γs, M.ϵ, M.k
    vv = γv*M.v + (1-γv)*∇f
    ss = γs*M.s + (1-γs)*∇f.*∇f
    # M.k = k += 1
    v_hat = vv ./ (1 - γv^k)
    s_hat = ss ./ (1 - γs^k)
    return x - α*v_hat ./ (sqrt.(s_hat) .+ ϵ)
end

step_without_update! (generic function with 2 methods)

#### Conjugate gradient descent
Credit: Yashuo

In [286]:
mutable struct ConjugateGD <: DescentMethod
    α
    g
    f
    d
    k # dumb variable
end
function init!(M::ConjugateGD)
    M.k = 0
    M.d = -M.g/sqrt(sum(M.g.^2))
end
# function step!(M::ConjugateGD, g, x, train_x, train_y)
function step!(M::ConjugateGD, g, x, train_x, train_y)
    d_prev, g_prev = M.d, M.g
    β = dot(transpose(g), g-g_prev)/dot(transpose(g_prev), g_prev)
    # β = dot(g, g-g_prev)/dot(g_prev, g_prev)
    β = max(0, β)
    d = -g + β*d_prev
    d = d/sqrt(sum(d.^2))
    M.α, dumb1, dumb2 = line_serach(M.f, g, x, train_x, train_y)
    x_next = x + M.α*d
    M.d, M.g = d, g
    return x_next
end
# function step_without_update!(M::ConjugateGD, g, x, train_x, train_y)
function step_without_update!(M::ConjugateGD, g, x, train_x, train_y)
    d_prev, g_prev = M.d, M.g
    β = dot(transpose(g), g-g_prev)/dot(transpose(g_prev), g_prev)
    # β = dot(g, g-g_prev)/dot(g_prev, g_prev)
    β = max(0, β)
    d = -g + β*d_prev
    d = d/sqrt(sum(d.^2))
    α, dumb1, dumb2 = line_serach(M.f, g, x, train_x, train_y)
    x_next = x + α*d
    # M.d, M.g = d, g
    return x_next
end
# # implementing strong backtracking line search
# function strong_backtracking_line_search(train_x, train_y, f, x, d, g; α=1, β=1e-4, σ=0.1, p=2)
#     if g == zeros(length(g))
#         # println("******************* g = 0 *******************")
#         return 0.0
#     end
#     # println("\tStart bracketing phase")
#     # bracketing phase
#     prev_α = 0
#     f_x, dumb1, dumb2, dumb3 = f(train_x, train_y, x)
#     x_αd = x+α*d
#     ∇f_x = transpose(g)*d
#     f_x_αd, hidden_outs, Y_pred, scores = f(train_x, train_y, x_αd) 
#     ∇_f_x_αd = ∇(train_x, train_y, _wrap(x_αd, params_shape), scores, Y_pred, hidden_outs)
#     ∇f_x_αd = transpose(_unwrap(∇_f_x_αd))*d
#     condition_a = f_x_αd >= f_x
#     condition_b = f_x_αd > f_x + β*α*∇f_x
#     condition_c = ∇f_x_αd >= 0
#     while !(condition_a || condition_b || condition_c)
#         prev_α, α = α, α*p
#         # println("\t\tα = $α")
#         x_αd = x+α*d
#         f_x_αd, hidden_outs, Y_pred, scores = f(train_x, train_y, x_αd) 
#         ∇_f_x_αd = ∇(train_x, train_y, _wrap(x_αd, params_shape), scores, Y_pred, hidden_outs)
#         ∇f_x_αd = transpose(_unwrap(∇_f_x_αd))*d
#         condition_a = f_x_αd >= f_x
#         condition_b = f_x_αd > f_x + β*α*∇f_x
#         condition_c = ∇f_x_αd >= 0
#     end
#     # println("\t bracket = [$prev_α, $α]")
#     # println("\tFinish bracketing phase")
#     # println("\tStart bracketing phase")
#     # zoom phase
#     # println("\t\tStart sufficient decrease condition")
#     # sufficient decrease condition
#     sufficient_decrease_condition = !condition_b
#     while !sufficient_decrease_condition
#         α = (prev_α + α)/2 
#         if α == 0.0
#             # println("******************* α = 0 *******************")
#             return 0.0
#         end
#         # println("\t\t\tα = $α")
#         x_αd = x+α*d
#         f_x_αd, hidden_outs, Y_pred, scores = f(train_x, train_y, x_αd) 
#         sufficient_decrease_condition = f_x_αd <= f_x + β*α*∇f_x
#     end
#     # println("\t\tFinish sufficient decrease condition")
#     # strong curvature condition 
#     # println("\t\tStart strong curvature condition")
#     ∇_f_x_αd = ∇(train_x, train_y, _wrap(x_αd, params_shape), scores, Y_pred, hidden_outs)
#     ∇f_x_αd = transpose(_unwrap(∇_f_x_αd))*d
#     strong_curvature_condition = abs(∇f_x_αd) <= -σ*∇f_x
#     while !strong_curvature_condition
#         α = (prev_α + α)/2
#         if α == 0.0
#             # println("******************* α = 0 *******************")
#             return 0.0
#         end
#         # println("\t\t\tα = $α")
#         x_αd = x+α*d
#         f_x_αd, hidden_outs, Y_pred, scores = f(train_x, train_y, x_αd) 
#         ∇_f_x_αd = ∇(train_x, train_y, _wrap(x_αd, params_shape), scores, Y_pred, hidden_outs)
#         ∇f_x_αd = transpose(_unwrap(∇_f_x_αd))*d
#         strong_curvature_condition = abs(∇f_x_αd) <= -σ*∇f_x
#     end
#     # println("\t\tFinish strong curvature condition")
#     # println("\tFinish bracketing phase")
#     return α
# end

step_without_update! (generic function with 8 methods)

#### Nesterov Momentum (SGD)
Credit: Dylan

In [21]:
# From K&W 76
mutable struct NesterovMomentum_SGD <: DescentMethod
    α # learning rate
    β # momentum decay
    f
    v # momentum
    k # dumb variable
end
function init!(M::NesterovMomentum_SGD, x_length)
    M.k = 0
    M.v = zeros(x_length)
end
function step!(M::NesterovMomentum_SGD, x, train_x, train_y)
    α, β, v = M.α, M.β, M.v
    x_new = x + β*v
    f_x_βv, hidden_outs, Y_pred, scores = M.f(train_x, train_y, x_new) 
    ∇f_x_βv = ∇(train_x, train_y, _wrap(x_new, params_shape), scores, Y_pred, hidden_outs)
    M.v = β*v - α*_unwrap(∇f_x_βv)
    return x + M.v
end
function step_without_update!(M::NesterovMomentum_SGD, x, train_x, train_y)
    α, β, v = M.α, M.β, M.v
    x_new = x + β*v
    f_x_βv, hidden_outs, Y_pred, scores = M.f(train_x, train_y, x_new) 
    ∇f_x_βv = ∇(train_x, train_y, _wrap(x_new, params_shape), scores, Y_pred, hidden_outs)
    vv = β*v - α*_unwrap(∇f_x_βv)
    return x + vv
end

step_without_update! (generic function with 4 methods)

#### Adadelta
Credit: Dylan

In [22]:
# source: K&W page 80
mutable struct Adadelta <: DescentMethod
    γs # gradient decay
    γx # update decay
    ϵ # small value
    s # sum of squared gradients
    u # sum of squared updates
    k # dumb variable
end
function init!(M::Adadelta, x_length)
    M.k = 0
    M.s = zeros(x_length)
    M.u = zeros(x_length)
    return M
end
function step!(M::Adadelta, g, x)
    γs, γx, ϵ, s, u = M.γs, M.γx, M.ϵ, M.s, M.u
    M.s = γs*s + (1-γs)*g.*g
    Δx = - (sqrt.(u) .+ ϵ) ./ (sqrt.(M.s) .+ ϵ) .* g
    M.u = γx*u + (1-γx)*Δx.*Δx
    return x + Δx
end
function step_without_update!(M::Adadelta, g, x)
    γs, γx, ϵ, s, u = M.γs, M.γx, M.ϵ, M.s, M.u
    ss = γs*s + (1-γs)*g.*g
    Δx = - (sqrt.(u) .+ ϵ) ./ (sqrt.(ss) .+ ϵ) .* g
    # u[:] = γx*u + (1-γx)*Δx.*Δx
    return x + Δx
end
    

step_without_update! (generic function with 5 methods)

#### Nelder-Mead
Credit: Yashuo

In [23]:
mutable struct Nelder_Mead <: DescentMethod
    f
    k # dumb variable
end
function init!(M::Nelder_Mead)
    M.k = 0
end
function step!(M::Nelder_Mead, x, train_x, train_y)
    return nmsmax(M.f, x, train_x, train_y)
end
function step_without_update!(M::Nelder_Mead, x, train_x, train_y)
    return nmsmax(M.f, x, train_x, train_y)
end
    

step_without_update! (generic function with 6 methods)

In [24]:
function nmsmax(fun, x, train_x, train_y; trace = true, initial_simplex = 0, target_f = Inf, max_its = Inf, max_evals = Inf, tol = 1e-3 )
    x0 = x[:];  # Work with column vector internally.
    n = length(x0);

   #  V = [zeros(n,1) eye(n)];
    V = [zeros(n,1) Matrix(1.0I, n, n)];
    f = zeros(n+1,1);
    V[:,1] = x0; 
    
    f[1], hidden_outs, Y_pred, scores = fun(train_x, train_y, x) 
   #  f[1] = fun(x);

    
    fmax_old = f[1];
    fmax     = -Inf; # Some initial value

   #  if trace
   #      @printf "f(x0) = %9.4e\n" f[1]
   #  end

    k = 0; m = 0;

    # Set up initial simplex.
    scale = max(norm(x0,Inf),1);
    if initial_simplex == 0
       # Regular simplex - all edges have same length.
       # Generated from construction given in reference [18, pp. 80-81] of [1].
       alpha = scale / (n*sqrt(2)) * [ sqrt(n+1)-1+n  sqrt(n+1)-1 ];
       V[:,2:n+1] = (x0 + alpha[2]*ones(n,1)) * ones(1,n);
       for j=2:n+1
           V[j-1,j] = x0[j-1] + alpha[1];
           x[:] = V[:,j]; 

           f[j], hidden_outs, Y_pred, scores = fun(train_x, train_y, x) 
         #   f[j] = fun(x);
       end
    else
       # Right-angled simplex based on co-ordinate axes.
       alpha = scale*ones(n+1,1);
       for j=2:n+1
           V[:,j] = x0 + alpha[j]*V[:,j];
           x[:] = V[:,j]; 
           f[j], hidden_outs, Y_pred, scores = fun(train_x, train_y, x) 
         #   f[j] = fun(x);
       end
    end
    nf = n+1;
    how = "initial  ";

    j = sortperm(f[:]);
    temp = f[j];
    j = j[n+1:-1:1];
    f = f[j]; V = V[:,j];

    alpha = 1;  beta = 1/2;  gamma = 2;

    msg = ""

    while true    ###### Outer (and only) loop.
    k = k+1;

        fmax = f[1];
      #   if fmax > fmax_old
      #       if trace
      #          @printf "Iter. %2.0f," k
      #          print(string("  how = ", how, " "));
      #          @printf "nf = %3.0f,  f = %9.4e  (%2.1f%%)\n" nf fmax 100*(fmax-fmax_old)/(abs(fmax_old)+eps(fmax_old));
      #       end
      #   end
        fmax_old = fmax;

        ### Three stopping tests from MDSMAX.M

        # Stopping Test 1 - f reached target value?
        if fmax >= target_f
           msg = "Exceeded target...quitting\n";
           break  # Quit.
        end

        # Stopping Test 2 - too many f-evals?
        if nf >= max_evals
           msg = "Max no. of function evaluations exceeded...quitting\n";
           break  # Quit.
        end

        # Stopping Test 3 - too many iterations?
        if k > max_its
           msg = "Max no. of iterations exceeded...quitting\n";
           break  # Quit.
        end

        # Stopping Test 4 - converged?   This is test (4.3) in [1].
        v1 = V[:,1];
        size_simplex = norm(V[:,2:n+1]-v1[:,ones(Int,n)],1) / max(1, norm(v1,1));
        if size_simplex <= tol
         #   msg = @sprintf("Simplex size %9.4e <= %9.4e...quitting\n", size_simplex, tol)
           break  # Quit.
        end

        #  One step of the Nelder-Mead simplex algorithm
        #  NJH: Altered function calls and changed CNT to NF.
        #       Changed each `fr < f[1]' type test to `>' for maximization
        #       and re-ordered function values after sort.

        temp = sum(V[:,1:n]'; dims = 1)
        vbar = (temp/n)';  # Mean value
        vr = (1 + alpha)*vbar - alpha*V[:,n+1]; x[:] = vr; 
        
        fr, hidden_outs, Y_pred, scores = fun(train_x, train_y, x) 
      #   fr = fun(x);

        nf = nf + 1;
        vk = vr;  fk = fr; how = "reflect, ";
        if fr > f[n]
                if fr > f[1]
                   ve = gamma*vr + (1-gamma)*vbar; x[:] = ve; 

                   fe, hidden_outs, Y_pred, scores = fun(train_x, train_y, x) 
                  #  fe = fun(x);

                   nf = nf + 1;
                   if fe > f[1]
                      vk = ve; fk = fe;
                      how = "expand,  ";
                   end
                end
        else
                vt = V[:,n+1]; ft = f[n+1];
                if fr > ft
                   vt = vr;  ft = fr;
                end
                vc = beta*vt + (1-beta)*vbar; x[:] = vc; 
                
                fc, hidden_outs, Y_pred, scores = fun(train_x, train_y, x) 
               #  fc = fun(x);

                nf = nf + 1;
                if fc > f[n]
                   vk = vc; fk = fc;
                   how = "contract,";
                else
                   for j = 2:n
                       V[:,j] = (V[:,1] + V[:,j])/2;
                       x[:] = V[:,j]; 
                       
                       f[j], hidden_outs, Y_pred, scores = fun(train_x, train_y, x) 
                     #   f[j] = fun(x);
                   end
                   nf = nf + n-1;
                   vk = (V[:,1] + V[:,n+1])/2; x[:] = vk; 

                   fk, hidden_outs, Y_pred, scores = fun(train_x, train_y, x) 
                  #  fk = fun(x);

                   nf = nf + 1;
                   how = "shrink,  ";
                end
        end
        V[:,n+1] = vk;
        f[n+1] = fk;
        j = sortperm(f[:]);
        temp = f[j];
        j = j[n+1:-1:1];
        f = f[j]; V = V[:,j];

    end   ###### End of outer (and only) loop.

    # Finished.
   #  if trace
   #      print(msg)
   #  end
    x[:] = V[:,1];

   #  return x, fmax, nf, k-1
    return x
end

nmsmax (generic function with 1 method)

#### LBFGS
Credit: David

In [296]:
mutable struct LBFGS <: DescentMethod
	n # number of variables
	f

    m # Memory length, was ∈ [2, 54] in paper
	prev_g # gradient at previous timestep 
	prev_x # x at previous timestep 
	Sm # previous m x's
	Ym # previous m gradients
	k # Internal iteration index
end

function init!(M::LBFGS)
    M.m = 20 
    M.prev_g = zeros(M.n)
    M.prev_x = zeros(M.n)
    M.Sm = zeros(M.n,M.m)
    M.Ym = zeros(M.n,M.m)
    M.k = 0 
end
function step!(o::LBFGS, ∇, x, train_x, train_y)
    # o.k += 1
	m = o.m
	
	gnorm = norm(∇)
	
	# if gnorm < τgrad # tolerance for the norm of the slope 
	# 	return; 
	# end
	
	s0 = x-o.prev_x
	y0 = ∇-o.prev_g
	
	# println("y0=$y0")
	H0 = s0'*y0/(y0'*y0) # hessian diagonal satisfying secant condition
    
    k = o.k
	# update Sm and Ym
	if k <= m
		o.Sm[:,k].=s0
		o.Ym[:,k].=y0
		p=-approxInvHess(∇,o.Sm[:,1:k],o.Ym[:,1:k],H0) 
	# only keep m entries in Sm and Ym so purge the old ones
		
	else
		o.Sm[:,1:(m-1)].=o.Sm[:,2:m]
		o.Ym[:,1:(m-1)].=o.Sm[:,2:m]
		o.Sm[:,m].=s0
		o.Ym[:,m].=y0
		p.=-approxInvHess(∇,o.Sm,o.Ym,H0)
	end
	
	# new direction=p, find new step size
    # α = 0.01
	α, fs, gs=line_serach(o.f, p, x, train_x, train_y)
	
	# update for next iteration
	o.prev_x = x
	o.prev_g = ∇
	x .= x + α.*p
    return x
	# f1=fs
	# g1=gs
	# k=k+1
	
	# if verbose == 1 
	# 	println("Iteration: $k -- x = $x1")
	# end
end
function step_without_update!(o::LBFGS, ∇, x, train_x, train_y)
    Smm = deepcopy(o.Sm)
    Ymm = deepcopy(o.Ym)

    # o.k += 1
	m = o.m
	
	gnorm = norm(∇)
	
	# if gnorm < τgrad # tolerance for the norm of the slope 
	# 	return; 
	# end
	
	s0 = x-o.prev_x
	y0 = ∇-o.prev_g
	
	# println("y0=$y0")
	H0 = s0'*y0/(y0'*y0) # hessian diagonal satisfying secant condition

    k = o.k
	# update Sm and Ym
	if k <= m
		Smm[:,k].=s0
		Ymm[:,k].=y0
		p=-approxInvHess(∇,Smm[:,1:k],Ymm[:,1:k],H0) 
	# only keep m entries in Sm and Ym so purge the old ones
		
	else
		Smm[:,1:(m-1)].=Smm[:,2:m]
		Ymm[:,1:(m-1)].=Smm[:,2:m]
		Smm[:,m].=s0
		Ymm[:,m].=y0
		p.=-approxInvHess(∇,Smm,Ymm,H0)
	end
	
	# new direction=p, find new step size
    # α = 0.01
	α, fs, gs=line_serach(o.f, p, x, train_x, train_y)
	
	# update for next iteration
	# o.prev_x = x
	# o.prev_g = ∇
	x .= x + α.*p
    return x
	# f1=fs
	# g1=gs
	# k=k+1
	
	# if verbose == 1 
	# 	println("Iteration: $k -- x = $x1")
	# end
end

function approxInvHess(g,S,Y,H0)
    #INPUT

    #g: gradient nx1 vector
    #S: nxk matrixs storing S[i]=x[i+1]-x[i]
    #Y: nxk matrixs storing Y[i]=g[i+1]-g[i]
    #H0: initial hessian diagnol scalar

    #OUTPUT
    # p:  the approximate inverse hessian multiplied by the gradient g
    #     which is the new direction
    #notation follows:
    #https://en.wikipedia.org/wiki/Limited-memory_BFGS

    n,k=size(S)
    rho=zeros(k)
    for i=1:k
        rho[i] = 1 /(Y[:,i]'*S[:,i])
        if rho[i]<0
            rho[i]=-rho[i]
        end
    end


    q=zeros(n,k+1)
    r=zeros(n,1)
    α=zeros(k,1)
    β=zeros(k,1)

    q[:,k+1]=g

    for i=k:-1:1
        α[i] =rho[i]*S[:,i]'*q[:,i+1]
        q[:,i].=q[:,i+1]-α[i]*Y[:,i]
    end

    z=zeros(size(q[:,1])[1])
    # println(size(H0))
    # println()
    # println(size(H0*q[:,1]))
    z.= H0*q[:,1]


    for i=1:k
        β[i] = rho[i]*Y[:,i]'*z
        z.=z+S[:,i]*(α[i]-β[i])
    end

    p=copy(z)

    return p
end

approxInvHess (generic function with 1 method)

# Start training

### Control variable for a scientifically correct comparison: **Give every optimizers a same NN to train**.

In [26]:
loss_ε = 0.01 # stop when loss <= loss_ε
# Initializing 
Y_pred, hidden_outs, Ws_copy = MLP(train_x_data[1], 3; hidden_layer_size = 16, num_hidden_layers = 2)
# println(Y_pred)
scores, losses, total_loss = softmax_loss(Y_pred, train_y_data[1])
∇_copy = ∇(train_x_data[1], train_y_data[1], Ws_copy, scores, Y_pred, hidden_outs);
println("New model created!\nInital loss = $total_loss")
params_shape = []
num_Ws = 0
for layer in Ws_copy
    # println(size(layer))
    s1, s2 = size(layer)
    num_Ws += s1 * s2
    push!(params_shape, size(layer))
end
function f(train_x, train_y, w_unwrapped)
    Y_pred, hidden_outs = MLP(train_x, 3; num_hidden_layers = 2, Ws=_wrap(w_unwrapped, params_shape), hidden_layer_size = 16)
    scores, losses, total_loss = softmax_loss(Y_pred, train_y)
    return total_loss, hidden_outs, Y_pred, scores
end

New model created!
Inital loss = 1.0677200789081591


f (generic function with 1 method)

## Optimizer = Steepest gradient descent

In [27]:
num_epoch_Steepest_GD = 20;

In [28]:
# Initializing 
optimizer_SteepestGD = SteepestGD(0.01, nothing)
init!(optimizer_SteepestGD)
Ws = deepcopy(Ws_copy);

In [29]:
# Train
for i = 1:num_epoch_Steepest_GD
    print("Epoch# $i\n\t")
    Ws, curr_loss = epoch_step(optimizer_SteepestGD, Ws; hidden_layer_size = 16)
    if curr_loss <= loss_ε
        break
    end
end
println("Finished")

Epoch# 1
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  1.0524875483449176
Epoch# 2
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  1.0085472899505192
Epoch# 3
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  1.03351415011928
Epoch# 4
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.964298680455558
Epoch# 5
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.9292304985643729
Epoch# 6
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.8800571011375639
Epoch# 7
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.8572806283191442
Epoch# 8
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.8548702865114243
Epoch# 9
	batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.8803701640133363
Epoch# 10
	ba

## Optimizer = Adam

In [30]:
num_epoch_Adam = 20;

In [31]:
# Initializing 
optimizer_Adam = Adam(0.01, 0.9, 0.999, 1e-7, nothing, nothing, nothing)
init!(optimizer_Adam, num_Ws);
Ws = deepcopy(Ws_copy);

In [32]:
# Train
for i = 1:num_epoch_Adam
    println("Epoch# $i")
    Ws, curr_loss = epoch_step(optimizer_Adam, Ws; hidden_layer_size = 16)
    if curr_loss <= loss_ε
        break
    end
end
println("Finished")

Epoch# 1
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.7489066868963872
Epoch# 2
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.290768542840515
Epoch# 3
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.015261841793336985
Epoch# 4
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  7.658779896820299e-5
Finished


## Optimizer = ConjugateGD

In [33]:
num_epoch_ConjugateGD = 20;
# function f(train_x, train_y, w_unwrapped)
#     Y_pred, hidden_outs = MLP(train_x, 3; num_hidden_layers = 2, Ws=_wrap(w_unwrapped, params_shape), hidden_layer_size = 16)
#     scores, losses, total_loss = softmax_loss(Y_pred, train_y)
#     return total_loss, hidden_outs, Y_pred, scores
# end

20

In [287]:
# Initializing 
# optimizer_ConjugateGD = ConjugateGD(0.01, _unwrap(deepcopy(∇_copy)), f, nothing, nothing)
optimizer_ConjugateGD = ConjugateGD(0.01, _unwrap(deepcopy(∇_copy)), f, nothing, nothing)
init!(optimizer_ConjugateGD);
Ws = deepcopy(Ws_copy);

In [288]:
# Train
for i = 1:num_epoch_ConjugateGD
    println("Epoch# $i")
    Ws, curr_loss = epoch_step(optimizer_ConjugateGD, Ws; hidden_layer_size = 16)
    if curr_loss <= loss_ε || isnan(curr_loss)
        break
    end
end
println("Finished")

Epoch# 1
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  1.0238170966475377
Epoch# 2
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  1.0178325729449416
Epoch# 3
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  1.0027438252035015
Epoch# 4
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.9745267630733193
Epoch# 5
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.8093880158735735
Epoch# 6
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.7089836926357035
Epoch# 7
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.5772751448216842
Epoch# 8
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.32247722613946095
Epoch# 9
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.06871508478345457
Epoch# 10
batch# 

## Optimizer = NesterovMomentum_SGD

In [36]:
num_epoch_NesterovMomentum_SGD = 20;

In [37]:
# Initializing 
optimizer_NesterovMomentum_SGD = NesterovMomentum_SGD(0.01, 0.99, f, nothing, nothing)
init!(optimizer_NesterovMomentum_SGD, num_Ws)
Ws = deepcopy(Ws_copy);

In [38]:
# Train
for i = 1:num_epoch_NesterovMomentum_SGD
    println("Epoch# $i")
    Ws, curr_loss = epoch_step(optimizer_NesterovMomentum_SGD, Ws; hidden_layer_size = 16)
    if curr_loss <= loss_ε || isnan(curr_loss)
        break
    end
end
println("Finished")

Epoch# 1
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.9909591715471994
Epoch# 2
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  1.2350562847293356
Epoch# 3
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.37973243651744487
Epoch# 4
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.1836650524263877
Epoch# 5
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.0031698707054810273
Finished


## Optimizer = Adadelta

In [39]:
num_epoch_Adadelta = 20;

In [80]:
# Initializing 
optimizer_Adadelta = Adadelta(0.95, 0.95, 1e-3, nothing, nothing, nothing)
init!(optimizer_Adadelta, num_Ws)
Ws = deepcopy(Ws_copy);

In [81]:
# Train
for i = 1:num_epoch_Adadelta
    println("Epoch# $i")
    Ws, curr_loss = epoch_step(optimizer_Adadelta, Ws; hidden_layer_size = 16)
    if curr_loss <= loss_ε || isnan(curr_loss)
        break
    end
end
println("Finished")

Epoch# 1
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  1.0289471623730455
Epoch# 2
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.8914206181512585
Epoch# 3
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.17906318944518004
Epoch# 4
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  0.005917156067241449
Finished


## Optimizer = Nelder-Mead

In [42]:
num_epoch_Nelder_Mead = 20;

In [43]:
# Initializing 
optimizer_Nelder_Mead = Nelder_Mead(f, nothing)
init!(optimizer_Nelder_Mead)
Ws = deepcopy(Ws_copy);

In [44]:
# # Train
# for i = 1:num_epoch_Nelder_Mead
#     println("Epoch# $i")
#     Ws, curr_loss = epoch_step(optimizer_Nelder_Mead, Ws; hidden_layer_size = 16)
#     if curr_loss <= loss_ε || isnan(curr_loss)
#         break
#     end
# end
# println("Finished")

## Optimizer = LBFGS

In [45]:
num_epoch_LBFGS = 20;

In [299]:
# Initializing 
optimizer_LBFGS = LBFGS(num_Ws, f, nothing, nothing, nothing, nothing, nothing, nothing)
init!(optimizer_LBFGS)
Ws = deepcopy(Ws_copy);

In [300]:
# Train
for i = 1:num_epoch_LBFGS
    println("Epoch# $i")
    Ws, curr_loss = epoch_step(optimizer_LBFGS, Ws; hidden_layer_size = 16)
    if curr_loss <= loss_ε || isnan(curr_loss)
        break
    end
end
println("Finished")

Epoch# 1
batch# 1 --> batch# 2 --> batch# 3 --> batch# 4 --> batch# 5 --> 
	train loss =  NaN
Finished
